In [ ]:
import pandas as pd

: 

In [ ]:
import polars as pl

splits = {'train': '1.0.0/train-*.parquet', 'validation': '1.0.0/validation-00000-of-00001.parquet', 'test': '1.0.0/test-00000-of-00001.parquet'}
df = pl.read_parquet('hf://datasets/abisee/cnn_dailymail/' + splits['train'])


: 

In [ ]:
df = df.drop(['id'])
df = pd.DataFrame(df)

: 

In [ ]:
for i in range(5):
    print("Article #", i + 1)
    print('Original Text')
    print(df.iloc[i, 0])

    print('\n\n Summary Text:')
    print(df.iloc[i, 1])

    print('==' * 250)

: 

In [8]:
dfn=pd.DataFrame()
dfn['article'] = df[0]
dfn['summary'] = df[1]


In [9]:
dfn.isnull().sum()
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   article  287113 non-null  object
 1   summary  287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


# Data preprocessing

In [10]:
# A list of contractions
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       287113 non-null  object
 1   1       287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [12]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   article  287113 non-null  object
 1   summary  287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


# Feature Engineering (Extractive Approach)

extracting important sentences using <br>
<b>Term Frequency (TF-IDF)</b><br>
<b>Sentence Position </b><br>
<b>Sentence length </b><br>
<b>Keyword Density</b><br>
<b>Named Entity Recognition (NER)</b><br>

#### Gyan ki baatein<br>
<b><i>Term Frequency (TF-IDF):  </b></i> Statistical measure used to evaluate importance of a word in document relative to coupus of documents , it gives out scores <br>

<b><i>Sentence Position</b></i> Sentences at beginning and at end of the document often carry significant weight , assigning higher scores based on their position <br>

<b><i>Sentence Length</b></i> as Longer sentences carry more info , so using length as a scoring factor<br>

<b><i>Keyword Density </b></i> using keyword density to identify sentences with high concentration of important words

<b><i>Named Entity Recognition (NER) </b></i> NER helps identify and categorize key entities like names , organization , locations etc , Using spaCy to perform NER<br>

#### Sequence to be applied
<b><i>Named Entity Recognition (NER)</b></i> 

<b><i>Term Frequency (TF-IDF)</b></i> 

<b><i>Sentence Position</b></i> 

<b><i>Keyword Density</b></i> 

<b><i>Sentence Length</b></i> 

the following code block cleans and then applies feature extraction procedure properly

Fatser Text preprocessing

In [15]:
import spacy
import re
import torch
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from torch.utils.data import Dataset , DataLoader

#load spacy with GPU (if available)
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm' , disable=['parser' ]) #keep NER , lemmatizer
nlp.add_pipe("sentencizer") #enable sentence boundary detection

#configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 256
MAX_SENTENCES = 30 #truncate long documents

#custom stopwords 
STOPWORDS = nlp.Defaults.stop_words | {"'s" , "n't" , "'m" , "'re"}

#--------------------------------------------------------------------------------
#GPU-Optimized text preprocessing
#--------------------------------------------------------------------------------
def preprocess_batch(texts):
    '''Process 100 documents at once on GPU'''
    cleaned = []
    for doc in nlp.pipe(texts , batch_size=10):
        sentences = []
        for sent in doc.sents:
            tokens=[
                token.lemma_.lower() for token in sent
                if not token.is_stop and token.lemma_ not in STOPWORDS
                and not token.is_punct
            ]
            if tokens:
                sentences.append(' '.join(tokens))
        if sentences:
            cleaned.append(sentences[:MAX_SENTENCES]) #TRUNCATE
    return cleaned

#--------------------------------------------------------------------------------------
#Feature Engineering (GPU Tensors)
#--------------------------------------------------------------------------------------
class FeatureExtractor:
    def __init__(self):
        self.global_freq = defaultdict(int)

    def fit(self , all_sentences):
        '''BUILD global vocabulary'''
        for sentences in tqdm(all_sentences):
            for sent in sentences:
                for token in sent.split():
                    self.global_freq[token] +=1

    def transform(self , sentences):
        '''Convert sentences to Pytorch feature tensors'''
        features = []
        for i , sent in enumerate(sentences):
            tokens = sent.split()
            if not tokens:
                continue

            #TF-IDF Calculation (GPU)
            tf = torch.tensor(
                [tokens.count(word)/len(tokens) for word in tokens],
                device=DEVICE
            )
            idf = torch.tensor(
                [np.log(len(sentences)/(self.global_freq[word]+1))for word in tokens],
                device=DEVICE
            )
            tfidf = (tf*idf).sum().item()

            #positional feature
            position = i / len(sentences)

            #keyword density (top 20% frequent words)
            keywords = {k for k,v in self.global_freq.items()
                        if v>np.percentile(list(self.global_freq.values()), 80)}
            
            keyword_density = sum(1 for w in tokens if w in keywords) / len(tokens)

            features.append([tfidf , position , keyword_density])
        return torch.tensor(features , device=DEVICE)

#------------------------------------------------------------------------------------------------
#Pytorch model architecture
#-------------------------------------------------------------------------------------------
class SummaryClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = torch.nn.Sequential(
            torch.nn.Linear(3,128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(128,1),
            torch.nn.Sigmoid()
        )

    def forward(self , x):
        return self.layer(x)
    

#----------------------------------------------------------------------------------
# Dataset Handling
#----------------------------------------------------------------------------------------
class SummaryDataset(Dataset):
    def __init__(self, texts, summaries):
        # Preprocess texts and summaries together
        self.texts = []
        self.summaries = []
        
        # Process in batches
        text_batches = [texts[i:i+1000] for i in range(0, len(texts), 1000)]
        summary_batches = [summaries[i:i+1000] for i in range(0, len(summaries), 1000)]
        
        for text_batch, summary_batch in zip(text_batches, summary_batches):
            preproc_texts = preprocess_batch(text_batch)
            preproc_summaries = preprocess_batch(summary_batch)
            
            # Filter out empty documents
            for t, s in zip(preproc_texts, preproc_summaries):
                if t and s:  # Only keep non-empty texts and summaries
                    self.texts.append(t)
                    self.summaries.append(s)
        
        # Build labels after alignment
        self.labels = []
        for doc, summary in zip(self.texts, self.summaries):
            doc_clean = [" ".join(s.split()) for s in doc]
            summary_clean = [" ".join(s.split()) for s in summary]
            self.labels.append([
                1 if sent in summary_clean else 0 
                for sent in doc_clean
            ])
        
        # Initialize feature extractor
        self.extractor = FeatureExtractor()
        self.extractor.fit(self.texts)
        
        # Debug
        print(f"Final dataset size: {len(self.texts)} texts, {len(self.labels)} labels")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        features = self.extractor.transform(self.texts[idx])
        labels = torch.tensor(self.labels[idx], device=DEVICE).float()
        return features, labels
    
#-------------------------------------------------------------------------------------
# Training loop
#-----------------------------------------------------------------------------------

def train(model , dataloader , epochs=250):
    model = model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = torch.nn.BCELoss()

    for epoch in range(epochs):
        for features , labels in tqdm(dataloader):
            optimizer.zero_grad()
            outputs = model(features).squeeze()
            labels = labels.squeeze()
            loss = loss_fn(outputs , labels.float())
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1} Loss: {loss.item():.4f}')







In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset('cnn_dailymail' , '3.0.0' , split='train')

df = pd.DataFrame(dataset)

sample_df = df.sample(n=1000 , random_state=42)

sample_df.to_csv('cnn_daily_sample.csv' , index=False)

In [16]:
# --------------------------------------------------
# Main Execution
# --------------------------------------------------
if __name__ == "__main__":
    # Load your dataset (example format)
    texts = [
        "Artificial Intelligence (AI) is the simulation of human intelligence in machines that can perform tasks like learning, reasoning, and problem-solving.",
        "Machine Learning (ML) is a subset of AI where systems learn from data to improve their performance without being explicitly programmed.",
        "Deep Learning (DL) is a subset of ML that uses neural networks with multiple layers to model complex patterns in data.",
        "Natural Language Processing (NLP) is a field of AI focused on enabling machines to understand and generate human language.",
        "Big Data refers to extremely large datasets that require advanced tools for storage, processing, and analysis."
    ]
    
    summaries = [
        "AI enables machines to mimic human intelligence, including learning and decision-making.",
        "ML allows computers to learn patterns from data and make predictions.",
        "DL uses deep neural networks to model complex data patterns.",
        "NLP helps machines understand and generate human language.",
        "Big Data deals with massive datasets that need specialized analysis tools."
    ]
    
    # Check lengths
    assert len(texts) == len(summaries), "Texts and summaries must have the same length!"
    
    # Create DataLoader
    dataset = SummaryDataset(texts, summaries)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # Initialize and train model
    model = SummaryClassifier()
    train(model, dataloader)
    
    # Save model
    torch.save(model.state_dict(), "summary_model.pt")

100%|██████████| 5/5 [00:00<?, ?it/s]


Final dataset size: 5 texts, 5 labels


100%|██████████| 1/1 [00:00<00:00,  2.90it/s]


Epoch 1 Loss: 0.6602


100%|██████████| 1/1 [00:00<00:00, 31.90it/s]


...

Epoch 248 Loss: 0.0135


100%|██████████| 1/1 [00:00<00:00, 39.43it/s]


Epoch 249 Loss: 0.0117


100%|██████████| 1/1 [00:00<00:00, 44.27it/s]

Epoch 250 Loss: 0.0144


In [23]:
def generate_summary(model, text, top_k=7):
    """
    Generate a summary for a given text using the trained model.
    Args:
        model: Trained SummaryClassifier model
        text: Input text to summarize (string)
        top_k: Number of sentences to include in summary (default=3)
    Returns:
        List of selected summary sentences
    """
    # Ensure model is in evaluation mode
    model.eval()
    
    # Preprocess the input text
    processed_text = preprocess_batch([text])[0]
    print(f"Number of processed sentences: {len(processed_text)}")
    
    # Create feature extractor and fit on the processed text
    extractor = FeatureExtractor()
    extractor.fit([processed_text])
    
    # Get features for the text
    with torch.no_grad():
        features = extractor.transform(processed_text)
        print(f"Features shape: {features.shape}")
        
        # Handle empty case
        if len(processed_text) == 0:
            return []
        
        # Ensure features are 2D
        if features.dim() == 1:
            features = features.unsqueeze(0)
        
        # Get model predictions
        predictions = model(features)
        print(f"Raw predictions shape: {predictions.shape}")
        
        # Ensure predictions are in the right shape
        if predictions.dim() == 0:
            predictions = predictions.unsqueeze(0)
        elif predictions.dim() == 2:
            predictions = predictions.squeeze(-1)
            
        print(f"Processed predictions shape: {predictions.shape}")
        
        # Ensure we don't try to get more sentences than we have
        top_k = min(top_k, len(processed_text))
        
        # Convert predictions to CPU for sorting if needed
        predictions = predictions.cpu()
        
        # Select top k sentences based on prediction scores
        if len(predictions.shape) == 0:
            return [processed_text[0]] if processed_text else []
            
        top_indices = torch.argsort(predictions, descending=True)[:top_k].tolist()
        
        # Sort indices to maintain original sentence order
        top_indices = sorted(top_indices)
        
        # Return original sentences in order
        return [processed_text[i] for i in top_indices]

def test_summarizer():
    """
    Test the trained summarizer on new text input.
    """
    try:
        # Load the trained model
        model = SummaryClassifier()
        model.load_state_dict(torch.load("summary_model.pt"))
        model = model.to(DEVICE)
        
        # Test text
        test_text = """
"Artificial Intelligence (AI) is the simulation of human intelligence in machines that can perform tasks like learning, reasoning, and problem-solving."
        """
        
        print("\nProcessing test text...")
        # Generate summary
        summary_sentences = generate_summary(model, test_text, top_k=2)
        
        print("\nOriginal Text:")
        print(test_text)
        print("\nGenerated Summary:")
        print(" ".join(summary_sentences))
        
    except Exception as e:
        print(f"\nError occurred during testing:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")
        raise

if __name__ == "__main__":
    test_summarizer()


Processing test text...
Number of processed sentences: 2


100%|██████████| 1/1 [00:00<00:00, 974.97it/s]

Features shape: torch.Size([1, 3])
Raw predictions shape: torch.Size([1, 1])
Processed predictions shape: torch.Size([1])

Original Text:

"Artificial Intelligence (AI) is the simulation of human intelligence in machines that can perform tasks like learning, reasoning, and problem-solving."
        

Generated Summary:

 artificial intelligence ai simulation human intelligence machine perform task like learning reasoning problem solving
